In [4]:
# conda install scikit-learn
# install from: http://docs.python-requests.org/en/master/
# to fix ssl installation of python packages issue
# curl https://bootstrap.pypa.io/get-pip.py | python3
# instructions to setup virtualenv
# https://www.tensorflow.org/install/install_mac
# pip3 install -r requirements.txt

import requests
import cv2
import scipy.misc
import numpy as np
import csv
import sklearn.model_selection as model
import tensorflow
from csv import DictReader
from urllib.request import urlopen
from matplotlib.pyplot import imshow
from fr_utils import *

In [5]:
dataset_x = []
dataset_y = []

spamreader = csv.DictReader(open("./datasets/jcpenney_com-ecommerce_sample.csv"))
for row in spamreader:
    if (row["name_title"] != "" and row["product_image_urls"] != ""):
        dataset_x.append(row["product_image_urls"])
        dataset_y.append(row["name_title"])
    
X = np.array(dataset_x)
y = np.array(dataset_y)

X_train, X_test, y_train, y_test = model.train_test_split(X, y, test_size = 0.3)